- Time to load 24 hours of data = 1 hr. 
- Time to compute 24 hours of CSD = <5 minutes. 
- Time to validate 24 hours of CSD = 2 hours, 20 min. 
    - Memory footprint tripled during computation. 
- Time to detect 24 hours of sharp waves = 4h
- Time to compute 24 hours of sharp wave properties = 24h
- 24 hour detection is not prohibitively expensive, provided that computation of sharp-wave properties can be parallelized


- TODO: Parallelize sharp wave property computation

## Imports and notebook definition

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd

In [5]:
from ecephys.data import paths, channel_groups
from ecephys.sglx_utils import load_timeseries
from ecephys.signal.csd import get_kcsd
from ecephys.signal.sharp_wave_ripples import detect_sharp_waves, get_durations, get_midpoints, get_sink_amplitudes, get_sink_integrals
from ecephys.helpers.utils import store_df_h5
import ecephys.plot as eplt

In [6]:
SUBJECT = "Doppio"
CONDITION = "BSL-6+18"
START_TIME = None
END_TIME = None

## Load the data

In [7]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
hpc_chans = channel_groups.hippocampus[SUBJECT]
bin_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lf.bin")

In [8]:
(time, hpc_lfps, fs) = load_timeseries(bin_path, hpc_chans, start_time=START_TIME, end_time=END_TIME)

nChan: 385, nFileSamp: 215716004


### Explore LFPs (optional)

In [11]:
eplt.lfp_explorer(time, hpc_lfps, chan_labels=hpc_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## Detect sharp waves

In [9]:
interestimate_distance = intersite_distance = 0.020
k = get_kcsd(hpc_lfps, gdx=interestimate_distance, do_lcurve=True)

Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0126
min lambda 1e-12
max lambda 0.0126
l-curve (all lambda):  0.23
Best lambda and R =  0.0039366356376800745 ,  0.23


In [10]:
hpc_csd = k.values('CSD')

In [11]:
sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]

In [12]:
spws = detect_sharp_waves(time, sr_csd)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 3))
plt.hist(spws.start_time, bins=int(time.max() / 10))
ax1.set_xticks([x * 3600 for x in range(24)])
ax1.set_xticklabels([str(x) for x in range(24)])
plt.show()

### Compute SPW properties

In [13]:
spws["duration"] = get_durations(spws)
spws["midpoint"] = get_midpoints(spws)

In [14]:
spws["sink_amplitude"] = get_sink_amplitudes(spws, time, sr_csd) * (1e-6) # Scale to mA/mm

In [15]:
spws["sink_integral"] = get_sink_integrals(spws, time, fs, sr_csd) * (1e-6) * (1e3) # Scale to mA * ms

### Export results

In [16]:
n_chans = hpc_lfps.shape[1]
ele_pos = np.linspace(0.0, (n_chans - 1) * intersite_distance, n_chans).reshape(n_chans, 1)
detection_threshold=2.5
boundary_threshold=1
minimum_duration=0.005

In [17]:
metadata = dict(
    csd_chans=hpc_chans,
    detection_chans=sr_chans,
    electrode_positions=k.ele_pos, 
    intersite_distance=intersite_distance,
    gdx=k.gdx,
    lambd = k.lambd,
    R = k.R,
    detect_states=["all"],
    detection_zscore_threshold=detection_threshold,
    boundary_zscore_threshold=boundary_threshold,
    minimum_duration=minimum_duration,
)
spws_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="sharp_waves.h5")
store_df_h5(spws_path, spws, **metadata)